In [1]:
import math
def f1(x):
    return x**3+4*x**2+16
def cos2x(x):
    return math.cos(2*x)

## Numerical integration

### [Riemann sum](https://en.wikipedia.org/wiki/Riemann_sum) (метод прямокутників)

In [2]:
def riemannLoss(a,b,m,n,h=None):
    if not h:
        h = (b-a)/n
    return m*h**2*(b-a)/24
def riemann(a,b,f,n):
    """
    Arguments:
    ----------
    a,b -- bounds of a segment
    f -- input function
    n -- number of segments
    Returns:
    --------
    res -- integral value on a certain segment
    """
    h = (b-a)/n
    res = sum(f(a+h*(i+0.5)) for i in range(n))*h    
    return res

In [3]:
print(riemann(0,1,math.sin,10))
print(riemann(0,1,f1,10))
print(riemann(0,1,math.exp,10))
print(riemann(0,3*math.pi/4,math.cos,25))
print(riemann(0,math.pi/4,cos2x,3))

0.45988929071851814
17.578749999999996
1.7175660864611277
0.7073685564123139
0.5057575799637313


### [Trapezoidal rule](https://en.wikipedia.org/wiki/Trapezoidal_rule) (метод трапецій)

In [4]:
def trapezoidalLoss(a,b,m,n,h=None):
    if not h:
        h = (a-b)/n
    return m * h**4*(b-a)/12
def trapezoidal(a,b,f,n):
    """
    Arguments:
    ----------
    a,b -- bounds of a segment
    f -- function
    n -- number of segments, has to be even
    Returns:
    --------
    res -- integral value on a certain segment
    """
    
    if n%2!=0: 
        return "n must be even"
    h,res = (b-a)/n,0
    
    X = [a+j*h for j in range(n)] #x0=a
    
    res += f(a)*(X[1]-a)*0.5
    res += sum(f(X[i])*(X[i+1]-X[i-1])*0.5 for i in range(1,n-1))
    res += f(b)*(b-X[n-1])*0.5
    
    return res

In [5]:
a,b,n = 0,1,3
h = (b-a)/n
X = [a + j*h for j in range(n+1)]
Y = [math.cos(X[i])+math.cos(X[i+1]) for i in range(n) ]

In [6]:
def trapezodialLoss(a,b,f,m,n,h=None):
    if not h:
        h = (b-a)/n
    return m * h**4*(b-a)/12
def trapezodial2(a,b,f,n,m=None):
    """
    Arguments:
    ----------
    a,b -- bounds of a segment
    f -- function
    n -- number of segments, has to be even
    m -- max(f''(x)) on [a;b]
    Returns:
    res -- integral value on a segment
    loss -- error rate
    """
    
    h = (b-a)/n    
    X = [a+j*h for j in range(n+1)]
    res = sum(f(X[i])+f(X[i+1]) for i in range(n))*0.5*h
    if m:
        return res, trapezodialLoss(a,b,f,m,n,h)
    else:
        return res

In [7]:
trapezodial2(0,1,math.sin,10,)

0.45931454885797635

In [8]:
trapezodial2(0,3*math.pi/4,math.cos,10,3*math.pi/4)

(0.7038324076935244, 0.0014258885134204036)

In [9]:
print(trapezoidal(0,1,math.sin,10))
print(trapezoidal(0,1,f1,12))
print(trapezoidal(0,1,math.exp,12))

0.380981857895228
15.912085262345677
1.5108644216412084


In [10]:
trapezoidal(0,3*math.pi/4,math.cos,100)

0.723337729148807

In [11]:
print(trapezoidalLoss(0,3*math.pi/4,3*math.pi/4,10))
trapezoidal(0,math.pi/4,cos2x,100)

0.0014258885134204036


0.49986635413811764

### [Simpson's rule](https://en.wikipedia.org/wiki/Simpson%27s_rule) - quadratic interpolation

In [12]:
def simpsonLoss(a,b,m,n, h=None):
    if not h:
        h = (b-a)/n
    return m*h**4*(b-a)/2880
def simpson(a,b,f): 
    """
    Arguments:
    ----------
    a,b -- bounds of a segment
    f -- function
    Returns:
    --------
    integral value at on a certain segment
    """
    
    return (b-a)*(f(a)+4*f((a+b)*0.5)+f(b))/6

In [13]:
print(simpson(0,1,math.sin))
print(simpson(0,1,f1))
print(simpson(0,1,math.exp))

0.4598621898707848
17.583333333333332
1.7188611518765928


In [14]:
simpson(0,math.pi/4,cos2x)

0.5011399387461052

In [15]:
simpson(0,3*math.pi/4,math.cos)

0.7161366279481727

In [43]:
class NumericalIntegration:
    
    def loss(self,a,b,m,n,param,h=None):
        """
        Arguments:
        a,b -- bonds of a segment
        m -- max(abs(f(n)(x))), x[a;b], f(n) - n-th derivative over x
        n -- number of segments
        param -- specifies loss
        h -- interval between consecutive subsegments
        """
        
        if not h:
            h = (b-a)/n
        if param == "riemann":
            return h**2*m*(b-a)/24
        elif param == "trapezoidal":
            return h**2 * m *(b-a)/12
        else: #simpson
            return h**4*m*(b-a)/2880
    
    def riemann(self,a,b,f,n,m=None):
        """
        Arguments:
         a,b -- bounds of a segment
         f -- input function
         n -- number of segments
        Returns:
         res -- integral value on a certain segment
        """
        h = (b-a)/n
        res = sum(f(a+h*(i+0.5)) for i in range(n))*h    

        if m is not None:
            return res, self.loss(a,b,m,n,"riemann",h)
        return res
    
    
    def trapezoidal(self,a,b,f,n,m=None):
        """
        Arguments:
        ----------
        a,b -- bounds of a segment
        f -- function
        n -- number of segments, has to be even
        m -- max(f''(x)) on [a;b]
        Returns:
        res -- integral value on a segment
        loss -- error rate
        """

        h = (b-a)/n    
        X = [a+j*h for j in range(n+1)]
        res = sum(f(X[i])+f(X[i+1]) for i in range(n))*0.5*h
        if m is not None:
            return res, self.loss(a,b,m,n,"trapezoidal",h)
        return res  
    
    
    def simpson(self,a,b,f,n,m=None): 
        """
        Arguments:
        ----------
        a,b -- bounds of a segment
        f -- function
        Returns:
        --------
        integral value at on a certain segment
        """
        h = (b-a)/n
        res = h/6 * sum(f(a+h*i) + 4*f(a+h*(i+0.5)) + f(a+h*(i+1)) for i in range(n))  
        if m:
            return res, self.loss(a,b,m,n,"simpson",h)
        return res


In [44]:
n = NumericalIntegration()
print(n.riemann(0,math.pi/4,cos2x,10,4))
print(n.trapezoidal(0,math.pi/4,cos2x,10,4))
print(n.simpson(0,math.pi/4,cos2x,10,16))

(0.5005144120713542, 0.0008074551218828077)
(0.4989714931771786, 0.0016149102437656153)
(0.5000001057732957, 1.6602630467951466e-07)


### [Monte-Carlo method](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%9C%D0%BE%D0%BD%D1%82%D0%B5-%D0%9A%D0%B0%D1%80%D0%BB%D0%BE#%D0%98%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%BE%D0%BC_%D0%9C%D0%BE%D0%BD%D1%82%D0%B5-%D0%9A%D0%B0%D1%80%D0%BB%D0%BE) using expected value

In [18]:
#splitting segment into N points
a,b,n=0,5,10
x = [a + ((b-a)/(n))*i for i in range(n)]
print(x,len(x))

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5] 10


In [19]:
def monteCarlo(a,b,f,n):
    """
    Arguments:
    ----------
    a,b -- bounds of a segment
    f -- function
    n -- number of segments
    Returns:
    --------
    res -- integral value on a certain segment
    """
    
    X = [a + ((b-a)/n)*i for i in range(n)]
    return (b-a)*sum(f(p) for p in X)/n

In [20]:
print(monteCarlo(0,1,math.sin,100))
print(monteCarlo(0,1,f1,20))
print(monteCarlo(0,1,math.exp,300))

0.4554865083873183
17.460625
1.7154196164130118


In [21]:
monteCarlo(0,math.pi/4,cos2x,100)

0.503916709936791